# Table of Contents

1. [Load Packages and Data](#paragraph1)
2. [Raw Data Analysis](#paragraph2)
    1. [EV Survey Data](#subparagraph1)

## Objective
* Determine if there are fundamental issues with the data.
* Do the features appear to be useful to answer the question.



## Load the Required Packages<a name="paragraph1"></a>

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
import numpy as np

from os import listdir
from os.path import isfile, join

import os 
from pathlib import Path

## Defined Functions
will be used in the later sections

In [ ]:
def seabornhist(size=(10,10), data=None, hue_c=None, y=None):
    fig, ax = plt.subplots(figsize=size)
    ax = sns.histplot(y=y, data= ev_ownership_sans_original_owners, hue = hue_c, ax=ax)
    plt.show()
    return

## Load the raw data from the pkl files

In [ ]:
#collect info about current working directory and the location of the raw data pickle files

wd = Path(os.getcwd())
parent_dir = wd.parent.absolute()


In [ ]:
#Data for the EV Vehicle list and attributes
filename = "ev_vehicle_data_raw.pkl"
filepath = f"{wd}\\{filename}"
ev_vehicle_data_raw = pd.read_pickle(filepath)

In [ ]:
#Data for the EV survey
filename = "ev_survey_output_raw.pkl"
filepath = f"{wd}\\{filename}"
ev_survey_output_raw = pd.read_pickle(filepath)

## Raw Data Analysis<a name="paragraph2"></a>
Now that we have loaded the raw data into one place, let's take a look at initial findings.

### EV Survey Data<a name="subparagraph1"></a>

In [ ]:
ev_survey_output_raw.info(verbose = True)

#### What percentage of null values is present in each column?

In [ ]:
print(100*ev_survey_output_raw.isnull().mean().sort_values(ascending=True))


#plot the missing values for visual reference
a = 100*ev_survey_output_raw.isnull().mean().sort_values(ascending=True)
plt.plot(a)
plt.show()

Observations:

* The satisfaction survey results share an almost identical number of missing entries
* The peak missing value comes from the work_charge_type
* The discontinuance data fortunately contains <1% missing entries.

In [ ]:
fig_dims = (9, 6)
fig, ax = plt.subplots(figsize=fig_dims)
sns.heatmap(ev_survey_output_raw.isnull(), 
            yticklabels=False, 
            cbar=False, 
            cmap='viridis')

## What is the ownership distribution of consumers for this study?

__Definition__: 
* Continued means the user had an EV previously and bought another.
* Original means the user is a first time EV owner.
* Discontinued means that the user decided to get rid of their EV and go to a gasoline vehicle.

In [ ]:
sns.set_theme
ax = sns.countplot(x = ev_survey_output_raw.discontinuance, data = ev_survey_output_raw)
plt.show()

### Age distribution of the EV owners in the study

In [ ]:
ax = sns.countplot(x = ev_survey_output_raw.surveyed_age, data = ev_survey_output_raw)
plt.show()

In [ ]:
sns.pointplot(ev_survey_output_raw.electric_range, ev_survey_output_raw.months_owned)

### Profile the raw data for the EV survey

In [ ]:
#profile = ProfileReport(ev_survey_output_raw, title="Pandas Profiling Report")

In [ ]:
#profile.to_widgets()

### Profile the raw data for the EV vehicles

In [ ]:
#profile = ProfileReport(ev_vehicle_data_raw, title="Pandas Profiling Report")

In [ ]:
#profile.to_widgets()

In [ ]:
ev_survey_output_raw.index[ev_survey_output_raw['months_owned'] == True].tolist()

## Data Munging Step (get the data in shape for EDA and modeling)

### Reduce the survey data to respondents who continued or discontinued use.
For the purposes of generating a predictive model, the users who are original owners need to be dropped as there is not a way to predict their next behavior.

In [ ]:
#Retain the data for the users who are known to have either discontinued EV ownership or are continued owners.

ev_ownership_sans_original_owners = ev_survey_output_raw.loc[(ev_survey_output_raw.discontinuance == 'Continued') |
                                                             (ev_survey_output_raw.discontinuance == 'Discontinued')]


In [ ]:
text = f"The raw dataset included {len(ev_survey_output_raw)} entries, after filtering the original owners there are now {len(ev_ownership_sans_original_owners)} entries."
print(text)

In [ ]:
ev_ownership_sans_original_owners.head()

In [ ]:
ev_ownership_sans_original_owners.household_income.value_counts()

In [ ]:
#_= pd.plotting.scatter_matrix(ev_ownership_sans_original_owners, c='y', figsize=[8,8], s=150, marker='D')
# Make a pairplot of the ev survey data
##sns.pairplot(ev_ownership_sans_original_owners, kind='scatter')

### Set the columns with text data to lowercase
This will help with inconsistency if the data entry wasn't done precisely

In [ ]:
for col in ev_ownership_sans_original_owners.columns:
    ev_ownership_sans_original_owners[col]= ev_ownership_sans_original_owners[col]\
                                                    .map(lambda x: x.lower() if isinstance(x,str) else x)
    

In [ ]:
#profile = ProfileReport(ev_ownership_sans_original_owners, title="Pandas Profiling Report")

In [ ]:
#profile.to_widgets()

### Split the attributes for the Year Make and Model of each vehicle in the survey into separate columns
This also gets the columns in the format needed to join to the table with the EV vehicle list

In [ ]:
models_makes = ev_ownership_sans_original_owners["year_make_model"].str.split(" ", n = 1, expand = True)

year         = models_makes[0].str.split(" ", n = 1, expand = True)[0]
make         = models_makes[1].str.split(" ", n = 1, expand = True)[0]
model        = models_makes[1].str.split(" ", n = 1, expand = True)[0:][1]

temp_my_make = pd.DataFrame(columns =['year','make', 'model'] )
temp_my_make['year']  = year
temp_my_make['year'].replace(' ', '')
temp_my_make['make']  = make
temp_my_make['year'].replace(' ', '')
temp_my_make['model'] = model
temp_my_make.head()

In [ ]:
#What are the unique vehicle models in the survey?
temp_my_make.model.unique()

In [ ]:
ev_ownership_sans_original_owners['model_year'] = temp_my_make.year
ev_ownership_sans_original_owners['Make']       = temp_my_make.make
ev_ownership_sans_original_owners['Model']      = temp_my_make.model

In [ ]:
ev_ownership_sans_original_owners.head()

In [ ]:
column_list = ev_ownership_sans_original_owners.columns.to_list()

In [ ]:
#Extract the current column list. From this list, restructure the columns to the desired target format in the next cell.
column_list

In [ ]:
# Reorganize them to the preferred order
column_list = ['response_id',
 'year_submitted_survey_2',
 'months_owned',
 'model_year',
 'Make',
 'Model',           
 'year_make_model',
 'electric_range',
 'ev_type',
 'discontinuance',
 'surveyed_age',
 'surveyed_gender',
 'dist_1',
 'trips_greater_200',
 'safety_satisfaction',
 'comfort_satisfaction',
 'refuel_recharge_satisfaction',
 'performance_satisfaction',
 'env_impact_satisfaction',
 'purch_price_satisfaction',
 'reliability_satisfaction',
 'range_satisfaction',
 'charge_access_satisfaction',
 'adas_satisfaction',
 'household_income',
 'home_charge_type',
 'work_charge_type',
 'highest_charge_used',
 'backup_vehicle_mpg',
 'finance_type',
 'last_page']

### Reindex the dataframe,  drop the year_make_model column now that it has been split into separate cols. 

In [ ]:
ev_ownership_sans_original_owners = ev_ownership_sans_original_owners.reindex(columns = column_list)
ev_ownership_sans_original_owners.drop(columns = ['year_make_model'], inplace=True)

In [ ]:
ev_ownership_sans_original_owners.head()

## Merging the datasets, this is a work in progress which may not work out becuase the set of vehicles is different in both.

In [ ]:
#What are the unique vehicle models in the vehicle data list?
ev_vehicle_data_raw.Model.unique()

In [ ]:
ev_vehicle_data_raw.head()

#### Join the two dataframes together, add information from the EV vehicles list to the entries in the survey data

In [ ]:
merged_ev_data = ev_ownership_sans_original_owners.merge(ev_vehicle_data_raw, on=['Make','Model'], how='left')

In [ ]:
merged_ev_data.info()

In [ ]:
merged_ev_data.head()

In [ ]:
missing = pd.concat([ev_ownership_sans_original_owners.isnull().sum(), \
                     100 * ev_ownership_sans_original_owners.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by= '%', ascending=False)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ax = sns.histplot(y='months_owned', data= ev_ownership_sans_original_owners, hue = 'finance_type', ax=ax)
plt.legend(title='Months Owned', loc='upper right', labels=['Lease', 'Own/Other'])
plt.show()

#Using the defined function
seabornhist(data=ev_ownership_sans_original_owners, hue_c= 'finance_type', y='months_owned')
seabornhist(data=ev_ownership_sans_original_owners, hue_c= 'work_charge_type', y='surveyed_age')

In [ ]:
ev_ownership_sans_original_owners.surveyed_age.value_counts()


In [ ]:
ev_ownership_sans_original_owners.trips_greater_200.value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ax = sns.countplot(y='adas_satisfaction', data= ev_ownership_sans_original_owners, hue = 'finance_type', ax=ax)
plt.legend(title='Months Owned', loc='upper right', labels=['Lease', 'Own/Other'])
plt.show()

### What is the distribution of makes in the dataset?

In [ ]:
sns.countplot(y=ev_ownership_sans_original_owners['Make'] , data = ev_ownership_sans_original_owners)

Looks good, no duplicate auto manufacturers

In [ ]:
ev_ownership_sans_original_owners[ev_ownership_sans_original_owners['surveyed_age']<=0]
ev_ownership_sans_original_owners[ev_ownership_sans_original_owners['months_owned']<0]
ev_ownership_sans_original_owners[ev_ownership_sans_original_owners['electric_range']<0]

#### How fuel efficient are the backup vehicles?

In [ ]:
fig_dims = (10, 15)
fig, ax = plt.subplots(figsize=fig_dims)
sns.countplot(y=ev_ownership_sans_original_owners.backup_vehicle_mpg, data=ev_ownership_sans_original_owners, ax=ax)

In [ ]:
ev_ownership_sans_original_owners.hist(figsize=(15, 10))
plt.subplots_adjust(hspace=0.5);

In [ ]:
#just to know how much data we lose if we drop all rows with na
ev_survey_no_nan = ev_ownership_sans_original_owners.dropna()

In [ ]:
ev_survey_no_nan.info()

## Drop other columns that wont be useful

In [ ]:
ev_ownership_sans_original_owners = ev_ownership_sans_original_owners.drop(columns= ['last_page'])

## Save the wrangled survey data as a .pkl for further use

In [ ]:
ev_ownership_sans_original_owners.to_pickle("ev_survey_wrangled_hasna.pkl")

In [ ]:
raw_output = pd.read_pickle("ev_survey_wrangled_hasna.pkl")

In [ ]:
raw_output.head()